# Imports

In [1]:
# %load_ext autoreload
# %autoreload 2

In [2]:
# %reload_ext autoreload

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import transforms
import time
from itertools import islice
from dataclasses import dataclass
import torchvision
from torchvision.models import densenet161, DenseNet161_Weights, vit_b_16, ViT_B_16_Weights, densenet121, DenseNet121_Weights
import os
import sys
from pathlib import Path
from torchinfo import summary

In [4]:
sys.path.append(str(Path.cwd().parent.parent))
from CheXpert.race_prediction.dataset import CheXpertRaceDataset
from CheXpert.shared_utils import vprint, to_gpu
from CheXpert import shared_utils
from CheXpert.race_prediction.utils import Configs, Mode

# Configs 

In [5]:
@dataclass
class TrainingConfigs(Configs):
    DATA_DIR = os.path.join("..", "..", "data", "CheXpert", "CheXpert-v1.0-small")
    DEMO_FILENAME = "CHEXPERT DEMO.csv"
    TRAIN_LABELS_FILENAME = "train.csv"
    VALID_LABELS_FILENAME = "valid.csv"
    CHECKPOINT_DIR = r"checkpoints"
    DISEASE_PRETRAINED_MODEL_PATH = os.path.join("..", "disease_prediction", "trained_models",
                                                 "2022_07_12-18_47__densenet121_aug__epoch-5__iter-12659__batch_size-16__trainLastLoss-0.3754__validAUC-0.8899.dict")
    BATCH_SIZE = 16
    EPOCHS = 10
    LEARNING_RATE = 1e-4
    LEARNING_RATE_REDUCE_PATIENCE = 3 # number of epochs with no improvement before reducing LR
    LEARNING_RATE_REDUCING_FACTOR = 0.5
    LEARNING_RATE_MIN_VAL = 1e-5
    CHECKPOINT_TIME_INTERVAL = 60*60 # seconds
    MODEL_VERSION = "densenet121_race_denseblock2_shallow"
    FREEZING_POINT = "classifier"
    SHALLOW_DENSEBLCOK = 1
    MODE = Mode.Shallow
    TRAINED_MODEL_PATH = None
    TRAIN_LOADER_SIZE = None
    VALID_LOADER_SIZE = None

In [6]:
shared_utils.set_seed(TrainingConfigs.SEED)

In [7]:
if torch.cuda.is_available():
    vprint(f"Memory info: {torch.cuda.mem_get_info()[0]/10e8:.1f} GB free GPU.", TrainingConfigs)
else: 
    vprint(f"No GPU Memory.", TrainingConfigs)

2022-07-20 08:55: Memory info: 8.5 GB free GPU.


# Training

## Training Setup

In [8]:
train_transform = transforms.Compose([
    transforms.Resize((320,320)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
    # augmentation
    transforms.RandomHorizontalFlip(p=0.25),
    transforms.RandomApply([transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.01)], p=0.1),
    transforms.RandomApply([torchvision.transforms.GaussianBlur(kernel_size=(3,3) ,sigma=(0.25, 0.75))], p=0.1),
    torchvision.transforms.RandomAdjustSharpness(sharpness_factor=0.75, p=0.1),
    torchvision.transforms.RandomAdjustSharpness(sharpness_factor=1.25, p=0.1),
])

valid_transform = transforms.Compose([
    transforms.Resize((320,320)),
    transforms.ToTensor(), 
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [9]:
# Create data loaders.
train_dataset = CheXpertRaceDataset(data_dir=TrainingConfigs.DATA_DIR, demo_filename=TrainingConfigs.DEMO_FILENAME,
                                       labels_filename=TrainingConfigs.TRAIN_LABELS_FILENAME,
                                       transform=train_transform)
train_dataloader = DataLoader(train_dataset, batch_size=TrainingConfigs.BATCH_SIZE, shuffle=False)
TrainingConfigs.TRAIN_LOADER_SIZE = len(train_dataloader)
len(train_dataset)

164248

In [10]:
valid_dataset = CheXpertRaceDataset(data_dir=TrainingConfigs.DATA_DIR, demo_filename=TrainingConfigs.DEMO_FILENAME,
                                       labels_filename=TrainingConfigs.VALID_LABELS_FILENAME,
                                       transform=valid_transform)
valid_dataloader = DataLoader(valid_dataset, batch_size=TrainingConfigs.BATCH_SIZE, shuffle=False)
TrainingConfigs.VALID_LOADER_SIZE = len(valid_dataloader)
len(valid_dataset)

172

In [33]:
model = densenet121(weights=DenseNet121_Weights.DEFAULT)
num_features = model.classifier.in_features
model.classifier = nn.Sequential(
    nn.Linear(num_features, num_features, bias=True),
    nn.ReLU(),
    nn.Dropout(p=0.1),
    nn.Linear(in_features=num_features, out_features=TrainingConfigs.NUM_DISEASE_CLASSES, bias=True)
)

In [34]:
if TrainingConfigs.MODE == Mode.PartlyFreezed:
    model = shared_utils.load_model(model, TrainingConfigs.DISEASE_PRETRAINED_MODEL_PATH)
if TrainingConfigs.MODE == Mode.Full or TrainingConfigs.MODE == Mode.PartlyFreezed:
    model.classifier[-1] = nn.Linear(in_features=num_features, out_features=TrainingConfigs.NUM_CLASSES, bias=True)

In [37]:
# {'training': True,
#  '_parameters': OrderedDict([('weight', Parameter containing:
#                tensor([ 2.3416e-02,  2.2803e-02,  2.4426e-02,  2.3758e-02, -4.1086e-02,
#                         4.8820e-02,  1.6445e-01, -2.0019e-02, -4.3458e-09,  2.5325e-02,
#                         3.7302e-02,  1.6524e-01, -1.9869e-02,  1.3128e-01,  5.7211e-02,
#                         2.3399e-08,  1.8768e-02, -1.4852e-01,  1.5488e-02,  1.9134e-02,
#                         1.7695e-02, -1.5466e-02,  2.3373e-02, -2.7411e-01, -2.1182e-02,
#                         9.3615e-02,  1.3569e-01, -2.0260e-02, -2.0156e-02,  1.9266e-02,
#                         2.2775e-01, -1.9635e-01,  4.6806e-02, -1.3008e-01,  2.7168e-02,
#                        -2.3198e-02,  1.8976e-02, -1.8244e-02,  2.5165e-01,  1.6805e-02,
#                         1.8075e-02, -2.7008e-01, -1.0888e-01,  1.9553e-02,  1.6578e-01,
#                         1.7743e-02,  2.2119e-02,  4.7337e-02, -2.0900e-02,  2.1206e-02,
#                        -4.8654e-02,  1.7948e-02,  2.3852e-02, -2.2607e-02,  1.8838e-02,
#                        -4.9182e-02,  2.2794e-02,  1.9441e-02,  2.5196e-02,  1.3313e-01,
#                         1.9131e-02,  2.4306e-02,  2.2703e-02,  1.3352e-01],
#                       requires_grad=True)),
# model.features[1]._parameters['weight']
model[1]._parameters['weight']

Parameter containing:
tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1.

In [36]:
if TrainingConfigs.MODE == Mode.Shallow:
    pretrained_model_state_dict = mode.state_dict()
    layer_offset = 3 + 2*TrainingConfigs.SHALLOW_DENSEBLCOK
    num_features = model.features[layer_offset].norm.num_features
    # TODO: pretrained on imagenet!
    model = nn.Sequential(model.features[:layer_offset],
                          nn.BatchNorm2d(num_features, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
                          nn.ReLU(),
                          nn.AdaptiveAvgPool2d((1, 1)),   
                          nn.Flatten(start_dim=1),
                          nn.Linear(in_features=num_features, out_features=num_features, bias=True),
                          nn.Dropout(p=0.1),
                          nn.Linear(in_features=num_features, out_features=TrainingConfigs.NUM_CLASSES, bias=True)
                     )
    pretrained_dict = {k: v for k, v in model.items() if k in model_dict}

In [38]:
pretrained_state = { k:v for k,v in pretrained_state.iteritems() if k in model_state and v.size() == model_state[k].size() }
model.state_dict()

OrderedDict([('0.conv0.weight',
              tensor([[[[ 7.8276e-02,  1.4949e-01,  1.6611e-01,  ...,  1.7676e-01,
                          1.6588e-01,  1.4101e-01],
                        [ 1.7546e-01,  2.4408e-01,  2.5000e-01,  ...,  2.7452e-01,
                          2.5245e-01,  2.2199e-01],
                        [ 1.2331e-01,  1.6441e-01,  1.4922e-01,  ...,  1.6301e-01,
                          1.6191e-01,  1.4061e-01],
                        ...,
                        [-1.0461e-01, -1.2065e-01, -1.1969e-01,  ..., -1.1355e-01,
                         -1.1181e-01, -1.1653e-01],
                        [-1.4747e-01, -1.8658e-01, -1.8272e-01,  ..., -2.1694e-01,
                         -2.0213e-01, -1.8302e-01],
                        [-2.0729e-01, -2.7118e-01, -2.8157e-01,  ..., -2.8711e-01,
                         -2.4883e-01, -2.2605e-01]],
              
                       [[ 1.6418e-01,  2.4814e-01,  2.6538e-01,  ...,  2.7358e-01,
                          2.56

In [15]:
# summary(model, input_size=[16, 3, 320, 320],
#         col_names=("input_size", "output_size", "num_params"))

In [16]:
if TrainingConfigs.MODE == Mode.PartlyFreezed:
    shared_utils.requires_grad_update_by_layer(model, TrainingConfigs, requires_grad=False)

In [17]:
optimizer = torch.optim.Adam(filter(lambda x: x.requires_grad, model.parameters()), lr=TrainingConfigs.LEARNING_RATE)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=TrainingConfigs.LEARNING_RATE_REDUCING_FACTOR,
                                                       patience=TrainingConfigs.LEARNING_RATE_REDUCE_PATIENCE, mode='min',
                                                       min_lr=TrainingConfigs.LEARNING_RATE_MIN_VAL)
criterion = nn.CrossEntropyLoss(reduction='mean')
# requires softmax to retrieve probabilities

## Training Loop 

In [18]:
checkpoint_obj = shared_utils.get_previous_training_place(model, optimizer, scheduler, criterion, TrainingConfigs)
model, optimizer, scheduler, criterion, results, last_epoch, last_iter = checkpoint_obj
score_dict = {
    "auc": "valid_auc",
    "loss": "valid_loss"
}
model.train()
model = to_gpu(model)
start_time = time.time()
shared_utils.start_training_msg(TrainingConfigs)
train_loss_list = []
apply_on_outputs = lambda x: torch.softmax(x, dim=1)
for epoch in range(last_epoch, TrainingConfigs.EPOCHS):
    train_dataloader_iter = tqdm(enumerate(train_dataloader), total=len(train_dataloader))
    if last_iter > -1:
        # fast foward dataloader
        train_dataloader_iter = islice(train_dataloader_iter, last_iter+1, len(train_dataloader))
        last_iter = -1
    for i, (images, labels) in train_dataloader_iter:
        images = to_gpu(images)
        labels = to_gpu(labels)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        train_loss_list.append(loss.item())
        if time.time()-start_time > TrainingConfigs.CHECKPOINT_TIME_INTERVAL:
            results['train_loss'].append(sum(train_loss_list)/len(train_loss_list))
            train_loss_list = []
            shared_utils.create_checkpoint(model, optimizer, scheduler, criterion, epoch, i, valid_dataloader,
                                           results, TrainingConfigs, score_dict, 
                                           apply_on_outputs=apply_on_outputs,
                                           by_study=None, challenge_ann_only=None)
            assert model.training
            start_time = time.time()
    shared_utils.create_checkpoint(model, optimizer, scheduler, criterion, epoch, len(train_dataloader), valid_dataloader, 
                                   results, TrainingConfigs, score_dict, apply_on_outputs=apply_on_outputs,
                                   by_study=None, challenge_ann_only=None)
    scheduler.step(results["valid_loss"][-1])

2022-07-20 08:55: 
2022-07-20 08:55: ----------------------------------------------------------------------------------------------------
2022-07-20 08:55: ----------------------------------------------------------------------------------------------------
2022-07-20 08:55: 
2022-07-20 08:55: Start Training


  0%|          | 0/10266 [00:00<?, ?it/s]

KeyboardInterrupt: 